In [73]:
import random
import statistics
from random import randint
import matplotlib
import matplotlib.pyplot as plt

In [74]:
population_size = 100
num_iterations = 20
averages = []
crossover_rate = 0.9
mutation_rate = 0.4
min_bins_per_problem = []

In [75]:
def create_initial_solution(items, capacity):
    #Initialise empty 'bins' list
    bins = []
    for weight, quantity in items.items():
        for _ in range(quantity):
            #Tracks if items have been placed in bins
            placed = False
            #Randomize bin selection to introduce diversity
            random.shuffle(bins)  
            for bin in bins:
                #Checks item weight plus current weight in bin against total capacity of bin
                if sum(weight * q for w, q in bin.items()) + weight <= capacity:
                    #if item's weight is already in the bin, increment by 1, otherwise add new item's weight
                    bin[weight] = bin.get(weight, 0) + 1
                    placed = True
                    break
            #new bin is created if item could not be placed in previous bins        
            if not placed:
                bins.append({weight: 1})
    return bins

In [76]:
#creates population of initial results and stores them in a list as potential solutions
def create_initial_population(items, capacity, population_size):
    return [create_initial_solution(items, capacity) for _ in range(population_size)]

In [77]:
def read_bin_packing_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        #Process the number of weights
        print(lines[1])
        print(lines[1].strip())
        num_weights = int(lines[0].strip())
        
        #Process the capacity of bins
        capacity = int(lines[1].strip())
        
        #Initialize the dictionary for items and their quantities
        items = {}
        
        #Process the items and quantities
        for line in lines[3:]:  
            #Start reading from the fourth line
            parts = line.strip().split()
            #Ensure the line has exactly two numbers
            if len(parts) == 2:  
                weight, quantity = int(parts[0]), int(parts[1])
                items[weight] = quantity
                
    return num_weights, capacity, items

In [78]:
# Example usage
file_paths = [
'C:\\Users\\brenn\\Downloads\\Binpacking-2.txt',
'C:\\Users\\brenn\\Downloads\\Binpacking-22.txt',
'C:\\Users\\brenn\\Downloads\\Binpacking-23.txt',
'C:\\Users\\brenn\\Downloads\\Binpacking-24.txt',
'C:\\Users\\brenn\\Downloads\\Binpacking-25.txt',  
]

# num_weights, capacity, items = read_bin_packing_file(file_path)

# print("Number of weights:", num_weights)
# print("Capacity:", capacity)
# print("Items:", items)

In [79]:
def fitness_function(solution, bin_capacity):
    
    #Count number of bins used in solution
    num_bins_used = len(solution)
    
    #Calculating amount of penalties for overfilling bins
    penalty = 0
    for bin in solution:
        total_weight = sum(weight * quantity for weight, quantity in bin.items())
        if total_weight > bin_capacity:
            #Applying penalty for overfilling bins
            penalty += (total_weight - bin_capacity)  
            
    #Calculating fitness based on the number of bins used and peanlties that were applied for overfilling bins
    fitness_score = 1 / (num_bins_used + penalty)
    
    
    return fitness_score

In [80]:
def crossover(parent1, parent2):
    #Ensure parents are not empty
    if random.randrange(0, 1) < crossover_rate:
        if not parent1 or not parent2:
              raise ValueError("Parents must contain at least one bin.")
    
    #Randomly select a crossover point
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    
    #Create offspring by combining the bins from both parents
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]

    offspring1 = [bin for bin in offspring1 if bin]
    offspring2 = [bin for bin in offspring2 if bin]

    return offspring1, offspring2

In [81]:
def mutate(solution, bin_capacity, mutation_rate=0.5):
    #Iterate through each bin in the solution
    for i, bin in enumerate(solution):
        if random.random() < mutation_rate:  # Apply mutation based on the mutation rate
            #Randomly select an item (weight) to move              
            item_weight = random.choice(list(bin.keys()))
            
            #Attempt to move the item to a different bin
            for j, other_bin in enumerate(solution):
                #Ensure it's a different bin
                if i != j:  
                    #Check if moving the item does not exceed the other bin's capacity
                    if sum(weight * quantity for weight, quantity in other_bin.items()) + item_weight <= bin_capacity:
                        #Move the item
                        other_bin[item_weight] = other_bin.get(item_weight, 0) + 1
                        bin[item_weight] -= 1
                        
                        #Remove the item from the original bin if its quantity reaches 0
                        if bin[item_weight] == 0:
                            del bin[item_weight]
                        break
            break  

In [82]:
def tournament_selection(population, scores, rounds=5):
    #Pick some random index in population
    valid_chrom_found = False
    while not valid_chrom_found:
        best_chrom = random.randint(0, population_size-1)
        if all(len(dct2.keys()) > 0 for dct2 in population[best_chrom]):
            valid_chrom_found = True
    
    #Pick contending chromosomes randomly
    for chrom in range(rounds+1):
        chrom_index = random.randint(0, population_size-1)
        
        #Find best score in tournament round
        if scores[chrom_index] > scores[best_chrom]:
            best_chrom = chrom
    return population[best_chrom]

In [83]:
def genetic_algorithm(population, capacity):
      for gen in range(num_iterations):
        parents = []
        children = []

        #Calculate fitness scores using fitness function
        scores = [fitness_function(x, capacity) for x in population]
        #Calculate averages
        averages.append(statistics.fmean(scores))
 
        #Find index and score of most fit in population
        best_index, best_score = scores.index(max(scores)), max(scores)

        #Print scores of current generation
        print(f'Generation {gen+1}')
        print(f'Best item: {population[best_index]}')
        print(f'Number of bins used in solution: {len(population[best_index])}')
        print(f'Item score: {best_score}')
        print(f'Generation average score: {averages[gen]}\n')
    
        #Find parent chromosomes of next generation via selection
        for i in range(population_size):
              parents.append(tournament_selection(population, scores))

        #Pair up chromosome with the one right after it
        for i in range(0, population_size, 2):
              parent_1, parent_2 = parents[i], parents[i+1]

        #Perform crossover, mutate children and append to list
        for child in crossover(parent_1, parent_2):
            #See if there's an empty dictionary in crossover
            child = mutate(child, capacity)
            children.append(child)
  
      #Replace current generation with children
      population = children

In [84]:
def main(items, capacity, population_size):
    population = create_initial_population(items, capacity, population_size)
    genetic_algorithm(population, capacity)

In [87]:
for file_path in file_paths:
    #create list to store numerical values
    numeric_lines = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                #convert numerical values into floating point number
                numeric_lines.append(float(line))
            except ValueError:
                #Skip non-numeric lines
                pass  
    num_weights, capacity, items = read_bin_packing_file(file_paths[i])
    print("Number of items for this problem", num_weights)
    print("Problem:", i)
    main(items, capacity, population_size)

    1000

1000
Number of items for this problem 46
Problem: 0
Generation 1
Best item: [{197: 1, 194: 2, 193: 2, 160: 1}, {151: 2, 150: 2, 200: 1, 166: 1}, {152: 3, 151: 3}, {178: 3, 177: 2, 159: 1}, {169: 1, 167: 1, 161: 1, 160: 1, 159: 1, 158: 1}, {171: 2, 170: 1, 169: 2, 162: 1}, {168: 2, 166: 3}, {195: 3, 176: 2}, {185: 1, 184: 1, 183: 1, 182: 1, 181: 1, 162: 1}, {154: 3, 153: 3}, {158: 2, 157: 3, 156: 1}, {199: 2, 198: 2, 197: 1, 165: 1}, {188: 1, 187: 2, 186: 2, 161: 1}, {189: 3, 188: 2, 161: 1}, {192: 4, 191: 1, 160: 1}, {175: 1, 174: 1, 173: 1, 172: 1, 171: 1, 162: 1}, {181: 2, 180: 1, 179: 2, 164: 1}, {200: 4, 196: 1}, {191: 1, 190: 4, 163: 1}, {156: 1, 155: 5}]
Number of bins used in solution: 20
Item score: 0.0010351966873706005
Generation average score: 0.000846107376947787

Generation 2
Best item: [{168: 2, 166: 2, 189: 1}, {189: 2, 188: 2, 162: 1}, {175: 1, 174: 1, 173: 1, 172: 1, 171: 1, 162: 1}, {188: 1, 187: 2, 186: 2, 161: 1}, {185: 1, 184: 1, 183: 1, 182: 1, 181: 1, 1